<a href="https://colab.research.google.com/github/mengphilshen/final-project-GoHawaii/blob/main/FourSquareAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

## Data Import

In [ ]:
# import modules
import pandas as pd
import requests
import folium

In [ ]:
# mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# helper function
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
# set credentials
client_id = 'Z4NHD5IUOAKVJFVGPM2QAHX4BKFQ3HBR3SDHKQVBC0DHPF5X'
client_secret = 'QVVE0H14JUI51DTBHJG3FSBXPR1YHR5QUB3U0FTCF3KDPWHC'
version = '20191231'

In [ ]:
# load Hawaii_neighbourhoods dataset
neighbourhoods0 = pd.read_csv("/content/drive/My Drive/CIS550_Project/Data/Airbnb_Hawaii/Hawaii_neighbourhoods.csv")

In [ ]:
neighbourhoods0

,neighbourhood_group,neighbourhood,latitude,longitude
0,Hawaii,Hamakua,19.34,-155.30
1,Hawaii,Kau,19.34,-155.30
2,Hawaii,North Hilo,19.34,-155.30
3,Hawaii,North Kohala,19.34,-155.30
4,Hawaii,North Kona,19.34,-155.30
5,Hawaii,Puna,19.34,-155.30
6,Hawaii,South Hilo,19.34,-155.30
7,Hawaii,South Kohala,19.34,-155.30
8,Hawaii,South Kona,19.34,-155.30
9,Honolulu,Central Oahu,21.28,-157.59


In [ ]:
# select unique values
neighbourhoods1 = neighbourhoods0.drop('neighbourhood', axis=1).drop_duplicates()
neighbourhoods1

,neighbourhood_group,latitude,longitude
0,Hawaii,19.34,-155.30
9,Honolulu,21.28,-157.59
17,Kauai,22.05,-159.30
23,Maui,20.48,-156.20


## Search by City

In [ ]:
# specify parameters
city = "Honolulu"

In [ ]:
# fetch data from API
url  = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={}&v={}'.format(client_id, client_secret, city, version)
results = requests.get(url).json()

In [ ]:
# extract data from JSON
venues = results['response']['groups'][0]['items']
venues[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4b05ababf964a520b7e022e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/touristinformation_',
     'suffix': '.png'},
    'id': '4f4530164b9074f6e4fb00ff',
    'name': 'Tourist Information Center',
    'pluralName': 'Tourist Information Centers',
    'primary': True,
    'shortName': 'Tourist Information'}],
  'id': '4b05ababf964a520b7e022e3',
  'location': {'address': '2270 Kalakaua Ave',
   'cc': 'US',
   'city': 'Honolulu',
   'country': 'United States',
   'crossStreet': 'Seaside Ave',
   'formattedAddress': ['2270 Kalakaua Ave (Seaside Ave)',
    'Honolulu, HI 96815',
    'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 21.278592972965924,
     'lng': -157.8278959377229}],
   'lat': 21.278592972965924,
   'lng': -157.8278959377229,
   'postalCode'

In [ ]:
# normalize JSON to dataframe
nearby0 = pd.json_normalize(venues)
nearby0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   referralId                           30 non-null     object 
 1   reasons.count                        30 non-null     int64  
 2   reasons.items                        30 non-null     object 
 3   venue.id                             30 non-null     object 
 4   venue.name                           30 non-null     object 
 5   venue.location.address               27 non-null     object 
 6   venue.location.crossStreet           13 non-null     object 
 7   venue.location.lat                   30 non-null     float64
 8   venue.location.lng                   30 non-null     float64
 9   venue.location.labeledLatLngs        29 non-null     object 
 10  venue.location.postalCode            29 non-null     object 
 11  venue.location.cc                 

In [ ]:
nearby0.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.location.neighborhood,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name,flags.outsideRadius
0,e-0-4b05ababf964a520b7e022e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b05ababf964a520b7e022e3,Hawaii Visitors & Convention Bureau,2270 Kalakaua Ave,Seaside Ave,21.278593,-157.827896,"[{'label': 'display', 'lat': 21.27859297296592...",96815,US,Honolulu,HI,United States,"[2270 Kalakaua Ave (Seaside Ave), Honolulu, HI...","[{'id': '4f4530164b9074f6e4fb00ff', 'name': 'T...",0,[],54880485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e-0-57c008f5498e258e1f1a6649-1,0,"[{'summary': 'This spot is popular', 'type': '...",57c008f5498e258e1f1a6649,Kona Coffee Purveyors | b.patisserie,2330 Kalakaua Ave,facing Kuhio Ave & Nahua St,21.279083,-157.825689,"[{'label': 'display', 'lat': 21.27908273226474...",96815,US,Honolulu,HI,United States,[2330 Kalakaua Ave (facing Kuhio Ave & Nahua S...,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN,Waikiki,1905882,https://www.grubhub.com/restaurant/kona-coffee...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
2,e-0-4af59763f964a5200efa21e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4af59763f964a5200efa21e3,Halekulani,2199 Kalia Rd,Lewers Street,21.278178,-157.832036,"[{'label': 'display', 'lat': 21.27817764241634...",96815,US,Honolulu,HI,United States,"[2199 Kalia Rd (Lewers Street), Honolulu, HI 9...","[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e-0-4b1d63b3f964a520d80f24e3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4b1d63b3f964a520d80f24e3,USS Arizona Memorial,1 Arizona Memorial Pl,NaN,21.364930,-157.950008,"[{'label': 'display', 'lat': 21.36493008797309...",96818,US,Honolulu,HI,United States,"[1 Arizona Memorial Pl, Honolulu, HI 96818, Un...","[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,e-0-4b64d3acf964a5201dd32ae3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b64d3acf964a5201dd32ae3,Lana'i Lookout,NaN,NaN,21.276488,-157.685543,"[{'label': 'display', 'lat': 21.27648750147807...",96825,US,Honolulu,HI,United States,"[Honolulu, HI 96825, United States]","[{'id': '4bf58dd8d48988d165941735', 'name': 'S...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [ ]:
# duplicate dataframe
nearby1 = nearby0.copy()

In [ ]:
# filter columns,consider only required columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in nearby1.columns if col.startswith('venue.location.')] + ['venue.id']
nearby2 = nearby1.loc[:, filtered_columns]
nearby2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   venue.name                       30 non-null     object 
 1   venue.categories                 30 non-null     object 
 2   venue.location.address           27 non-null     object 
 3   venue.location.crossStreet       13 non-null     object 
 4   venue.location.lat               30 non-null     float64
 5   venue.location.lng               30 non-null     float64
 6   venue.location.labeledLatLngs    29 non-null     object 
 7   venue.location.postalCode        29 non-null     object 
 8   venue.location.cc                30 non-null     object 
 9   venue.location.city              30 non-null     object 
 10  venue.location.state             30 non-null     object 
 11  venue.location.country           30 non-null     object 
 12  venue.location.formatted

In [ ]:
# filter the category for each row
nearby2['venue.categories'] = nearby2.apply(get_category_type, axis=1)

In [ ]:
# clean columns
nearby2.columns = [col.split('.')[-1] for col in nearby2.columns]
nearby2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              30 non-null     object 
 1   categories        30 non-null     object 
 2   address           27 non-null     object 
 3   crossStreet       13 non-null     object 
 4   lat               30 non-null     float64
 5   lng               30 non-null     float64
 6   labeledLatLngs    29 non-null     object 
 7   postalCode        29 non-null     object 
 8   cc                30 non-null     object 
 9   city              30 non-null     object 
 10  state             30 non-null     object 
 11  country           30 non-null     object 
 12  formattedAddress  30 non-null     object 
 13  neighborhood      5 non-null      object 
 14  id                30 non-null     object 
dtypes: float64(2), object(13)
memory usage: 3.6+ KB


In [ ]:
nearby2.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Hawaii Visitors & Convention Bureau,Tourist Information Center,2270 Kalakaua Ave,Seaside Ave,21.278593,-157.827896,"[{'label': 'display', 'lat': 21.27859297296592...",96815,US,Honolulu,HI,United States,"[2270 Kalakaua Ave (Seaside Ave), Honolulu, HI...",NaN,4b05ababf964a520b7e022e3
1,Kona Coffee Purveyors | b.patisserie,Café,2330 Kalakaua Ave,facing Kuhio Ave & Nahua St,21.279083,-157.825689,"[{'label': 'display', 'lat': 21.27908273226474...",96815,US,Honolulu,HI,United States,[2330 Kalakaua Ave (facing Kuhio Ave & Nahua S...,Waikiki,57c008f5498e258e1f1a6649
2,Halekulani,Hotel,2199 Kalia Rd,Lewers Street,21.278178,-157.832036,"[{'label': 'display', 'lat': 21.27817764241634...",96815,US,Honolulu,HI,United States,"[2199 Kalia Rd (Lewers Street), Honolulu, HI 9...",NaN,4af59763f964a5200efa21e3
3,USS Arizona Memorial,Historic Site,1 Arizona Memorial Pl,NaN,21.364930,-157.950008,"[{'label': 'display', 'lat': 21.36493008797309...",96818,US,Honolulu,HI,United States,"[1 Arizona Memorial Pl, Honolulu, HI 96818, Un...",NaN,4b1d63b3f964a520d80f24e3
4,Lana'i Lookout,Scenic Lookout,NaN,NaN,21.276488,-157.685543,"[{'label': 'display', 'lat': 21.27648750147807...",96825,US,Honolulu,HI,United States,"[Honolulu, HI 96825, United States]",NaN,4b64d3acf964a5201dd32ae3


## Search by Location

In [ ]:
# helper function
def get_nearby_venues(latitude, longitude, radius, limit):
  # fetch data from API
  url  = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(client_id, client_secret, latitude, longitude, version, radius, limit)
  results = requests.get(url).json()
  # extract data from JSON
  venues = results['response']['groups'][0]['items']
  return venues

### Hawaii

In [ ]:
# fetch data from API
venues_hawaii = get_nearby_venues(19.34, -155.30, 60000, 5000)

In [ ]:
venues_hawaii[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4be894da88ed2d7ff911cc1d-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
     'suffix': '.png'},
    'id': '52e81612bcbc57f1066b7a21',
    'name': 'National Park',
    'pluralName': 'National Parks',
    'primary': True,
    'shortName': 'National Park'}],
  'id': '4be894da88ed2d7ff911cc1d',
  'location': {'address': '1 Crater Rim Dr',
   'cc': 'US',
   'city': 'Volcano',
   'country': 'United States',
   'distance': 10898,
   'formattedAddress': ['1 Crater Rim Dr',
    'Volcano, HI 96718',
    'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 19.429178614915262,
     'lng': -155.25717583721595}],
   'lat': 19.429178614915262,
   'lng': -155.25717583721595,
   'postalCode': '96718',
   'state': 'HI'},
  'name': "Hawai'i Volcanoes N

In [ ]:
# normalize JSON to dataframe
nearby_hawaii0 = pd.json_normalize(venues_maui)
nearby_hawaii0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   referralId                       14 non-null     object 
 1   reasons.count                    14 non-null     int64  
 2   reasons.items                    14 non-null     object 
 3   venue.id                         14 non-null     object 
 4   venue.name                       14 non-null     object 
 5   venue.location.lat               14 non-null     float64
 6   venue.location.lng               14 non-null     float64
 7   venue.location.labeledLatLngs    14 non-null     object 
 8   venue.location.distance          14 non-null     int64  
 9   venue.location.postalCode        12 non-null     object 
 10  venue.location.cc                14 non-null     object 
 11  venue.location.city              12 non-null     object 
 12  venue.location.state    

In [ ]:
# duplicate dataframe
nearby_hawaii1 = nearby_hawaii0.copy()

In [ ]:
# filter columns,consider only required columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories'] + [col for col in nearby_hawaii1.columns if col.startswith('venue.location.')]
nearby_hawaii2 = nearby_hawaii1.loc[:, filtered_columns]
nearby_hawaii2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   venue.id                         14 non-null     object 
 1   venue.name                       14 non-null     object 
 2   venue.categories                 14 non-null     object 
 3   venue.location.lat               14 non-null     float64
 4   venue.location.lng               14 non-null     float64
 5   venue.location.labeledLatLngs    14 non-null     object 
 6   venue.location.distance          14 non-null     int64  
 7   venue.location.postalCode        12 non-null     object 
 8   venue.location.cc                14 non-null     object 
 9   venue.location.city              12 non-null     object 
 10  venue.location.state             12 non-null     object 
 11  venue.location.country           14 non-null     object 
 12  venue.location.formatted

In [ ]:
# add an island column
nearby_hawaii2['venue.island'] = 'Hawaii'
nearby_hawaii2['venue.county'] = 'Hawaii'
nearby_hawaii2.head()

,venue.id,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.address,venue.location.crossStreet,venue.island,venue.county
0,4fc565dbe4b0829fe1507460,Lelekea Beach,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",20.651087,-156.079459,"[{'label': 'display', 'lat': 20.651087, 'lng':...",22815,96713,US,Hāna,HI,United States,"[Hāna, HI 96713, United States]",NaN,NaN,Hawaii,Hawaii
1,50f5e39ee4b0255d39ffc198,Haukini Bay,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",20.624844,-156.210364,"[{'label': 'display', 'lat': 20.62484351951365...",16159,96790,US,Kula,HI,United States,"[Kula, HI 96790, United States]",NaN,NaN,Hawaii,Hawaii
2,4e4e09abbd4101d0d7a0c8b7,Cold Stone Creamery,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",20.444210,-156.163382,"[{'label': 'display', 'lat': 20.44421, 'lng': ...",5518,96740,US,Kailua-Kona,HI,United States,"[75-1027 Henry St, Kailua-Kona, HI 96740, Unit...",75-1027 Henry St,NaN,Hawaii,Hawaii
3,501a833fe4b0a724be572f34,Alenuihaha Channel,"[{'id': '4bf58dd8d48988d12d951735', 'name': 'B...",20.390015,-156.246970,"[{'label': 'display', 'lat': 20.39001493552374...",11151,NaN,US,NaN,NaN,United States,[United States],NaN,NaN,Hawaii,Hawaii
4,57eb1a3d498e11e2a0e8c6a4,Pokowai Sea Arch,"[{'id': '4bf58dd8d48988d165941735', 'name': 'S...",20.622090,-156.213070,"[{'label': 'display', 'lat': 20.62209, 'lng': ...",15875,96790,US,Kula,HI,United States,"[Kula, HI 96790, United States]",NaN,NaN,Hawaii,Hawaii


### Oahu

In [ ]:
# fetch data from API
venues_oahu = get_nearby_venues(21.28, -157.59, 25000, 5000)

In [ ]:
venues_oahu[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4b9c5ab1f964a5201d6236e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/lighthouse_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d15d941735',
    'name': 'Lighthouse',
    'pluralName': 'Lighthouses',
    'primary': True,
    'shortName': 'Lighthouse'}],
  'id': '4b9c5ab1f964a5201d6236e3',
  'location': {'address': 'Makapuu Lighthouse Rd',
   'cc': 'US',
   'city': 'Honolulu',
   'country': 'United States',
   'distance': 7123,
   'formattedAddress': ['Makapuu Lighthouse Rd',
    'Honolulu, HI 96825',
    'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 21.30921836573366,
     'lng': -157.65110493534797}],
   'lat': 21.30921836573366,
   'lng': -157.65110493534797,
   'postalCode': '96825',
   'state': 'HI'},
  'name': 'Makapu‘u Lighthous

In [ ]:
# normalize JSON to dataframe
nearby_oahu0 = pd.json_normalize(venues_oahu)
nearby_oahu0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   referralId                           100 non-null    object 
 1   reasons.count                        100 non-null    int64  
 2   reasons.items                        100 non-null    object 
 3   venue.id                             100 non-null    object 
 4   venue.name                           100 non-null    object 
 5   venue.location.address               88 non-null     object 
 6   venue.location.lat                   100 non-null    float64
 7   venue.location.lng                   100 non-null    float64
 8   venue.location.labeledLatLngs        99 non-null     object 
 9   venue.location.distance              100 non-null    int64  
 10  venue.location.postalCode            91 non-null     object 
 11  venue.location.cc                

In [ ]:
nearby_oahu0.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.location.neighborhood,venue.venuePage.id,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name
0,e-0-4b9c5ab1f964a5201d6236e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b9c5ab1f964a5201d6236e3,Makapu‘u Lighthouse,Makapuu Lighthouse Rd,21.309218,-157.651105,"[{'label': 'display', 'lat': 21.30921836573366...",7123,96825,US,Honolulu,HI,United States,"[Makapuu Lighthouse Rd, Honolulu, HI 96825, Un...","[{'id': '4bf58dd8d48988d15d941735', 'name': 'L...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e-0-4e5135f26284416669a6dd71-1,0,"[{'summary': 'This spot is popular', 'type': '...",4e5135f26284416669a6dd71,Makapu'u Point Lighthouse Trail,NaN,21.305453,-157.655084,"[{'label': 'display', 'lat': 21.30545310149249...",7321,96825,US,Honolulu,HI,United States,"[Honolulu, HI 96825, United States]","[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",0,[],within Ka Iwi State Scenic Shoreline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e-0-4bf1ca3852bda5936efab1b7-2,0,"[{'summary': 'This spot is popular', 'type': '...",4bf1ca3852bda5936efab1b7,Makapu‘u Beach Park (Makapuu Beach Park),41-095 Kalanianaole Hwy,21.313181,-157.661057,"[{'label': 'display', 'lat': 21.31318090753942...",8243,96795,US,Waimānalo,HI,United States,"[41-095 Kalanianaole Hwy, Waimānalo, HI 96795,...","[{'id': '4bf58dd8d48988d1e3941735', 'name': 'S...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e-0-4b367db6f964a520a33625e3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4b367db6f964a520a33625e3,Makapu‘u Lookout,41-10001 Hawaii 72,21.309118,-157.657171,"[{'label': 'display', 'lat': 21.30911776117945...",7683,NaN,US,Waimānalo,HI,United States,"[41-10001 Hawaii 72, Waimānalo, HI, United Sta...","[{'id': '4bf58dd8d48988d165941735', 'name': 'S...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e-0-4b3675cff964a520f53525e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b3675cff964a520f53525e3,Sandy Beach Park,8808 Kalanianaole Hwy,21.286018,-157.672158,"[{'label': 'display', 'lat': 21.28601791634979...",8548,96825,US,Honolulu,HI,United States,"[8808 Kalanianaole Hwy, Honolulu, HI 96825, Un...","[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# duplicate dataframe
nearby_oahu1 = nearby_oahu0.copy()

In [ ]:
# filter columns,consider only required columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories'] + [col for col in nearby_oahu1.columns if col.startswith('venue.location.')]
nearby_oahu2 = nearby_oahu1.loc[:, filtered_columns]
nearby_oahu2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   venue.id                         100 non-null    object 
 1   venue.name                       100 non-null    object 
 2   venue.categories                 100 non-null    object 
 3   venue.location.address           88 non-null     object 
 4   venue.location.lat               100 non-null    float64
 5   venue.location.lng               100 non-null    float64
 6   venue.location.labeledLatLngs    99 non-null     object 
 7   venue.location.distance          100 non-null    int64  
 8   venue.location.postalCode        91 non-null     object 
 9   venue.location.cc                100 non-null    object 
 10  venue.location.city              98 non-null     object 
 11  venue.location.state             100 non-null    object 
 12  venue.location.country 

In [ ]:
# add an island column
nearby_oahu2['venue.island'] = 'Oahu'
nearby_oahu2['venue.county'] = 'Honolulu'
nearby_oahu2.head()

,venue.id,venue.name,venue.categories,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.crossStreet,venue.location.neighborhood,venue.island,venue.county
0,4b9c5ab1f964a5201d6236e3,Makapu‘u Lighthouse,"[{'id': '4bf58dd8d48988d15d941735', 'name': 'L...",Makapuu Lighthouse Rd,21.309218,-157.651105,"[{'label': 'display', 'lat': 21.30921836573366...",7123,96825,US,Honolulu,HI,United States,"[Makapuu Lighthouse Rd, Honolulu, HI 96825, Un...",NaN,NaN,Oahu,Honolulu
1,4e5135f26284416669a6dd71,Makapu'u Point Lighthouse Trail,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",NaN,21.305453,-157.655084,"[{'label': 'display', 'lat': 21.30545310149249...",7321,96825,US,Honolulu,HI,United States,"[Honolulu, HI 96825, United States]",within Ka Iwi State Scenic Shoreline,NaN,Oahu,Honolulu
2,4bf1ca3852bda5936efab1b7,Makapu‘u Beach Park (Makapuu Beach Park),"[{'id': '4bf58dd8d48988d1e3941735', 'name': 'S...",41-095 Kalanianaole Hwy,21.313181,-157.661057,"[{'label': 'display', 'lat': 21.31318090753942...",8243,96795,US,Waimānalo,HI,United States,"[41-095 Kalanianaole Hwy, Waimānalo, HI 96795,...",NaN,NaN,Oahu,Honolulu
3,4b367db6f964a520a33625e3,Makapu‘u Lookout,"[{'id': '4bf58dd8d48988d165941735', 'name': 'S...",41-10001 Hawaii 72,21.309118,-157.657171,"[{'label': 'display', 'lat': 21.30911776117945...",7683,NaN,US,Waimānalo,HI,United States,"[41-10001 Hawaii 72, Waimānalo, HI, United Sta...",NaN,NaN,Oahu,Honolulu
4,4b3675cff964a520f53525e3,Sandy Beach Park,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",8808 Kalanianaole Hwy,21.286018,-157.672158,"[{'label': 'display', 'lat': 21.28601791634979...",8548,96825,US,Honolulu,HI,United States,"[8808 Kalanianaole Hwy, Honolulu, HI 96825, Un...",NaN,NaN,Oahu,Honolulu


### Maui

In [ ]:
# fetch data from API
venues_maui = get_nearby_venues(20.48, -156.20, 25000, 5000)

In [ ]:
venues_maui[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4fc565dbe4b0829fe1507460-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/beach_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1e2941735',
    'name': 'Beach',
    'pluralName': 'Beaches',
    'primary': True,
    'shortName': 'Beach'}],
  'id': '4fc565dbe4b0829fe1507460',
  'location': {'cc': 'US',
   'city': 'Hāna',
   'country': 'United States',
   'distance': 22815,
   'formattedAddress': ['Hāna, HI 96713', 'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 20.651087,
     'lng': -156.079459}],
   'lat': 20.651087,
   'lng': -156.079459,
   'postalCode': '96713',
   'state': 'HI'},
  'name': 'Lelekea Beach',
  'photos': {'count': 0, 'groups': []}}}

In [ ]:
# normalize JSON to dataframe
nearby_maui0 = pd.json_normalize(venues_maui)
nearby_maui0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   referralId                       14 non-null     object 
 1   reasons.count                    14 non-null     int64  
 2   reasons.items                    14 non-null     object 
 3   venue.id                         14 non-null     object 
 4   venue.name                       14 non-null     object 
 5   venue.location.lat               14 non-null     float64
 6   venue.location.lng               14 non-null     float64
 7   venue.location.labeledLatLngs    14 non-null     object 
 8   venue.location.distance          14 non-null     int64  
 9   venue.location.postalCode        12 non-null     object 
 10  venue.location.cc                14 non-null     object 
 11  venue.location.city              12 non-null     object 
 12  venue.location.state    

In [ ]:
nearby_maui0.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.address,venue.location.crossStreet
0,e-0-4fc565dbe4b0829fe1507460-0,0,"[{'summary': 'This spot is popular', 'type': '...",4fc565dbe4b0829fe1507460,Lelekea Beach,20.651087,-156.079459,"[{'label': 'display', 'lat': 20.651087, 'lng':...",22815,96713,US,Hāna,HI,United States,"[Hāna, HI 96713, United States]","[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",0,[],NaN,NaN
1,e-0-50f5e39ee4b0255d39ffc198-1,0,"[{'summary': 'This spot is popular', 'type': '...",50f5e39ee4b0255d39ffc198,Haukini Bay,20.624844,-156.210364,"[{'label': 'display', 'lat': 20.62484351951365...",16159,96790,US,Kula,HI,United States,"[Kula, HI 96790, United States]","[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",0,[],NaN,NaN
2,e-0-4e4e09abbd4101d0d7a0c8b7-2,0,"[{'summary': 'This spot is popular', 'type': '...",4e4e09abbd4101d0d7a0c8b7,Cold Stone Creamery,20.444210,-156.163382,"[{'label': 'display', 'lat': 20.44421, 'lng': ...",5518,96740,US,Kailua-Kona,HI,United States,"[75-1027 Henry St, Kailua-Kona, HI 96740, Unit...","[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",0,[],75-1027 Henry St,NaN
3,e-0-501a833fe4b0a724be572f34-3,0,"[{'summary': 'This spot is popular', 'type': '...",501a833fe4b0a724be572f34,Alenuihaha Channel,20.390015,-156.246970,"[{'label': 'display', 'lat': 20.39001493552374...",11151,NaN,US,NaN,NaN,United States,[United States],"[{'id': '4bf58dd8d48988d12d951735', 'name': 'B...",0,[],NaN,NaN
4,e-0-57eb1a3d498e11e2a0e8c6a4-4,0,"[{'summary': 'This spot is popular', 'type': '...",57eb1a3d498e11e2a0e8c6a4,Pokowai Sea Arch,20.622090,-156.213070,"[{'label': 'display', 'lat': 20.62209, 'lng': ...",15875,96790,US,Kula,HI,United States,"[Kula, HI 96790, United States]","[{'id': '4bf58dd8d48988d165941735', 'name': 'S...",0,[],NaN,NaN


In [ ]:
# duplicate dataframe
nearby_maui1 = nearby_maui0.copy()

In [ ]:
# filter columns,consider only required columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories'] + [col for col in nearby_maui1.columns if col.startswith('venue.location.')]
nearby_maui2 = nearby_maui1.loc[:, filtered_columns]
nearby_maui2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   venue.id                         14 non-null     object 
 1   venue.name                       14 non-null     object 
 2   venue.categories                 14 non-null     object 
 3   venue.location.lat               14 non-null     float64
 4   venue.location.lng               14 non-null     float64
 5   venue.location.labeledLatLngs    14 non-null     object 
 6   venue.location.distance          14 non-null     int64  
 7   venue.location.postalCode        12 non-null     object 
 8   venue.location.cc                14 non-null     object 
 9   venue.location.city              12 non-null     object 
 10  venue.location.state             12 non-null     object 
 11  venue.location.country           14 non-null     object 
 12  venue.location.formatted

In [ ]:
# add an island column
nearby_maui2['venue.island'] = 'Maui'
nearby_maui2['venue.county'] = 'Maui'
nearby_maui2.head()

,venue.id,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.address,venue.location.crossStreet,venue.island,venue.county
0,4fc565dbe4b0829fe1507460,Lelekea Beach,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",20.651087,-156.079459,"[{'label': 'display', 'lat': 20.651087, 'lng':...",22815,96713,US,Hāna,HI,United States,"[Hāna, HI 96713, United States]",NaN,NaN,Maui,Maui
1,50f5e39ee4b0255d39ffc198,Haukini Bay,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",20.624844,-156.210364,"[{'label': 'display', 'lat': 20.62484351951365...",16159,96790,US,Kula,HI,United States,"[Kula, HI 96790, United States]",NaN,NaN,Maui,Maui
2,4e4e09abbd4101d0d7a0c8b7,Cold Stone Creamery,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",20.444210,-156.163382,"[{'label': 'display', 'lat': 20.44421, 'lng': ...",5518,96740,US,Kailua-Kona,HI,United States,"[75-1027 Henry St, Kailua-Kona, HI 96740, Unit...",75-1027 Henry St,NaN,Maui,Maui
3,501a833fe4b0a724be572f34,Alenuihaha Channel,"[{'id': '4bf58dd8d48988d12d951735', 'name': 'B...",20.390015,-156.246970,"[{'label': 'display', 'lat': 20.39001493552374...",11151,NaN,US,NaN,NaN,United States,[United States],NaN,NaN,Maui,Maui
4,57eb1a3d498e11e2a0e8c6a4,Pokowai Sea Arch,"[{'id': '4bf58dd8d48988d165941735', 'name': 'S...",20.622090,-156.213070,"[{'label': 'display', 'lat': 20.62209, 'lng': ...",15875,96790,US,Kula,HI,United States,"[Kula, HI 96790, United States]",NaN,NaN,Maui,Maui


### Molokai

In [ ]:
# fetch data from API
venues_molokai = get_nearby_venues(21.08, -157.02, 20000, 5000)

In [ ]:
venues_molokai[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4b848120f964a520343931e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1c9941735',
    'name': 'Ice Cream Shop',
    'pluralName': 'Ice Cream Shops',
    'primary': True,
    'shortName': 'Ice Cream'}],
  'id': '4b848120f964a520343931e3',
  'location': {'cc': 'US',
   'city': 'Kaunakakai',
   'country': 'United States',
   'distance': 1086,
   'formattedAddress': ['Kaunakakai, HI 96748', 'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 21.089757251262945,
     'lng': -157.0196646186083}],
   'lat': 21.089757251262945,
   'lng': -157.0196646186083,
   'postalCode': '96748',
   'state': 'HI'},
  'name': "Dave's Hawaiian Ice Cream",
  'photos': {'count': 0, 'groups': []}}}

In [ ]:
# normalize JSON to dataframe
nearby_molokai0 = pd.json_normalize(venues_molokai)
nearby_molokai0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   referralId                       27 non-null     object 
 1   reasons.count                    27 non-null     int64  
 2   reasons.items                    27 non-null     object 
 3   venue.id                         27 non-null     object 
 4   venue.name                       27 non-null     object 
 5   venue.location.lat               27 non-null     float64
 6   venue.location.lng               27 non-null     float64
 7   venue.location.labeledLatLngs    26 non-null     object 
 8   venue.location.distance          27 non-null     int64  
 9   venue.location.postalCode        22 non-null     object 
 10  venue.location.cc                27 non-null     object 
 11  venue.location.city              26 non-null     object 
 12  venue.location.state    

In [ ]:
nearby_molokai0.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.address,venue.location.crossStreet,venue.venuePage.id
0,e-0-4b848120f964a520343931e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b848120f964a520343931e3,Dave's Hawaiian Ice Cream,21.089757,-157.019665,"[{'label': 'display', 'lat': 21.08975725126294...",1086,96748,US,Kaunakakai,HI,United States,"[Kaunakakai, HI 96748, United States]","[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",0,[],NaN,NaN,NaN
1,e-0-4b915333f964a5203cb333e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b915333f964a5203cb333e3,Kanemitsu Bakery,21.090829,-157.020902,"[{'label': 'display', 'lat': 21.09082868301856...",1209,96748,US,Kaunakakai,HI,United States,"[79 Ala Malama St., Kaunakakai, HI 96748, Unit...","[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",0,[],79 Ala Malama St.,NaN,NaN
2,e-0-4b537d0cf964a5207d9f27e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b537d0cf964a5207d9f27e3,Kualapuu Cookhouse,21.154996,-157.038020,"[{'label': 'display', 'lat': 21.15499562889081...",8555,96757,US,Kualapu‘u,HI,United States,"[Kualapu‘u, HI 96757, United States]","[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",0,[],NaN,NaN,NaN
3,e-0-4bfb1e3ed2b720a12fa2326a-3,0,"[{'summary': 'This spot is popular', 'type': '...",4bfb1e3ed2b720a12fa2326a,Hotel Moloka'i,21.078730,-156.996023,"[{'label': 'display', 'lat': 21.07873031325953...",2494,96748,US,Kaunakakai,HI,United States,"[1300 Kamehameha V Highway, #1020 (Near Mile M...","[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",0,[],"1300 Kamehameha V Highway, #1020",Near Mile Marker 2,NaN
4,e-0-4bc2a5ce4cdfc9b6fb709621-4,0,"[{'summary': 'This spot is popular', 'type': '...",4bc2a5ce4cdfc9b6fb709621,Molokai Burger,21.089247,-157.022309,"[{'label': 'display', 'lat': 21.08924731929179...",1056,96748,US,Kaunakakai,HI,United States,[20 W. Kamehameha V Highway (Kaunakakai Place)...,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",0,[],20 W. Kamehameha V Highway,Kaunakakai Place,NaN


In [ ]:
# duplicate dataframe
nearby_molokai1 = nearby_molokai0.copy()

In [ ]:
# filter columns,consider only required columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories'] + [col for col in nearby_molokai1.columns if col.startswith('venue.location.')]
nearby_molokai2 = nearby_molokai1.loc[:, filtered_columns]
nearby_molokai2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   venue.id                         27 non-null     object 
 1   venue.name                       27 non-null     object 
 2   venue.categories                 27 non-null     object 
 3   venue.location.lat               27 non-null     float64
 4   venue.location.lng               27 non-null     float64
 5   venue.location.labeledLatLngs    26 non-null     object 
 6   venue.location.distance          27 non-null     int64  
 7   venue.location.postalCode        22 non-null     object 
 8   venue.location.cc                27 non-null     object 
 9   venue.location.city              26 non-null     object 
 10  venue.location.state             27 non-null     object 
 11  venue.location.country           27 non-null     object 
 12  venue.location.formatted

In [ ]:
# add an island column
nearby_molokai2['venue.island'] = 'Molokai'
nearby_molokai2['venue.county'] = 'Maui'
nearby_molokai2.head()

,venue.id,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.address,venue.location.crossStreet,venue.island,venue.county
0,4b848120f964a520343931e3,Dave's Hawaiian Ice Cream,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",21.089757,-157.019665,"[{'label': 'display', 'lat': 21.08975725126294...",1086,96748,US,Kaunakakai,HI,United States,"[Kaunakakai, HI 96748, United States]",NaN,NaN,Molokai,Maui
1,4b915333f964a5203cb333e3,Kanemitsu Bakery,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",21.090829,-157.020902,"[{'label': 'display', 'lat': 21.09082868301856...",1209,96748,US,Kaunakakai,HI,United States,"[79 Ala Malama St., Kaunakakai, HI 96748, Unit...",79 Ala Malama St.,NaN,Molokai,Maui
2,4b537d0cf964a5207d9f27e3,Kualapuu Cookhouse,"[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",21.154996,-157.038020,"[{'label': 'display', 'lat': 21.15499562889081...",8555,96757,US,Kualapu‘u,HI,United States,"[Kualapu‘u, HI 96757, United States]",NaN,NaN,Molokai,Maui
3,4bfb1e3ed2b720a12fa2326a,Hotel Moloka'i,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",21.078730,-156.996023,"[{'label': 'display', 'lat': 21.07873031325953...",2494,96748,US,Kaunakakai,HI,United States,"[1300 Kamehameha V Highway, #1020 (Near Mile M...","1300 Kamehameha V Highway, #1020",Near Mile Marker 2,Molokai,Maui
4,4bc2a5ce4cdfc9b6fb709621,Molokai Burger,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",21.089247,-157.022309,"[{'label': 'display', 'lat': 21.08924731929179...",1056,96748,US,Kaunakakai,HI,United States,[20 W. Kamehameha V Highway (Kaunakakai Place)...,20 W. Kamehameha V Highway,Kaunakakai Place,Molokai,Maui


### Lanai

In [ ]:
# fetch data from API
venues_lanai = get_nearby_venues(20.50, -156.56, 15000, 5000)

In [ ]:
venues_lanai[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-5640a4d4cd10c8b4e41fce79-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d11f941735',
    'name': 'Nightclub',
    'pluralName': 'Nightclubs',
    'primary': True,
    'shortName': 'Nightclub'}],
  'id': '5640a4d4cd10c8b4e41fce79',
  'location': {'cc': 'US',
   'city': 'Kula',
   'country': 'United States',
   'distance': 5986,
   'formattedAddress': ['Kula, HI', 'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 20.538158416748047,
     'lng': -156.6004638671875}],
   'lat': 20.538158416748047,
   'lng': -156.6004638671875,
   'state': 'HI'},
  'name': "Ner'de",
  'photos': {'count': 0, 'groups': []}}}

In [ ]:
# normalize JSON to dataframe
nearby_lanai0 = pd.json_normalize(venues_lanai)
nearby_lanai0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   referralId                       4 non-null      object 
 1   reasons.count                    4 non-null      int64  
 2   reasons.items                    4 non-null      object 
 3   venue.id                         4 non-null      object 
 4   venue.name                       4 non-null      object 
 5   venue.location.lat               4 non-null      float64
 6   venue.location.lng               4 non-null      float64
 7   venue.location.labeledLatLngs    4 non-null      object 
 8   venue.location.distance          4 non-null      int64  
 9   venue.location.cc                4 non-null      object 
 10  venue.location.city              3 non-null      object 
 11  venue.location.state             3 non-null      object 
 12  venue.location.country    

In [ ]:
nearby_lanai0.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.address
0,e-0-5640a4d4cd10c8b4e41fce79-0,0,"[{'summary': 'This spot is popular', 'type': '...",5640a4d4cd10c8b4e41fce79,Ner'de,20.538158,-156.600464,"[{'label': 'display', 'lat': 20.53815841674804...",5986,US,Kula,HI,United States,"[Kula, HI, United States]","[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",0,[],NaN
1,e-0-567f4d6e498e44dfa296a880-1,0,"[{'summary': 'This spot is popular', 'type': '...",567f4d6e498e44dfa296a880,Whatever u want,20.553896,-156.588600,"[{'label': 'display', 'lat': 20.553896, 'lng':...",6699,US,Kula,HI,United States,"[Somewhere Over The Rainbow, Kula, HI, United ...","[{'id': '4bf58dd8d48988d1f2931735', 'name': 'P...",0,[],Somewhere Over The Rainbow
2,e-0-5640a46fcd10e06a4187cb35-2,0,"[{'summary': 'This spot is popular', 'type': '...",5640a46fcd10e06a4187cb35,Ner'de,20.553667,-156.603470,"[{'label': 'display', 'lat': 20.55366706848144...",7498,US,Kula,HI,United States,"[Kula, HI, United States]","[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",0,[],NaN
3,e-0-4f6c4eb8e4b07a4bc3461d71-3,0,"[{'summary': 'This spot is popular', 'type': '...",4f6c4eb8e4b07a4bc3461d71,Kapolei HI,20.483599,-156.417175,"[{'label': 'display', 'lat': 20.48359870910644...",15004,US,NaN,NaN,United States,[United States],"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",0,[],NaN


In [ ]:
# duplicate dataframe
nearby_lanai1 = nearby_lanai0.copy()

In [ ]:
# filter columns,consider only required columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories'] + [col for col in nearby_lanai1.columns if col.startswith('venue.location.')]
nearby_lanai2 = nearby_lanai1.loc[:, filtered_columns]
nearby_lanai2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   venue.id                         4 non-null      object 
 1   venue.name                       4 non-null      object 
 2   venue.categories                 4 non-null      object 
 3   venue.location.lat               4 non-null      float64
 4   venue.location.lng               4 non-null      float64
 5   venue.location.labeledLatLngs    4 non-null      object 
 6   venue.location.distance          4 non-null      int64  
 7   venue.location.cc                4 non-null      object 
 8   venue.location.city              3 non-null      object 
 9   venue.location.state             3 non-null      object 
 10  venue.location.country           4 non-null      object 
 11  venue.location.formattedAddress  4 non-null      object 
 12  venue.location.address    

In [ ]:
# add an island column
nearby_lanai2['venue.island'] = 'Lanai'
nearby_lanai2['venue.county'] = 'Maui'
nearby_lanai2.head()

,venue.id,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.address,venue.island,venue.county
0,5640a4d4cd10c8b4e41fce79,Ner'de,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",20.538158,-156.600464,"[{'label': 'display', 'lat': 20.53815841674804...",5986,US,Kula,HI,United States,"[Kula, HI, United States]",NaN,Lanai,Maui
1,567f4d6e498e44dfa296a880,Whatever u want,"[{'id': '4bf58dd8d48988d1f2931735', 'name': 'P...",20.553896,-156.588600,"[{'label': 'display', 'lat': 20.553896, 'lng':...",6699,US,Kula,HI,United States,"[Somewhere Over The Rainbow, Kula, HI, United ...",Somewhere Over The Rainbow,Lanai,Maui
2,5640a46fcd10e06a4187cb35,Ner'de,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",20.553667,-156.603470,"[{'label': 'display', 'lat': 20.55366706848144...",7498,US,Kula,HI,United States,"[Kula, HI, United States]",NaN,Lanai,Maui
3,4f6c4eb8e4b07a4bc3461d71,Kapolei HI,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",20.483599,-156.417175,"[{'label': 'display', 'lat': 20.48359870910644...",15004,US,NaN,NaN,United States,[United States],NaN,Lanai,Maui


### Kauai

In [ ]:
# fetch data from API
venues_kauai = get_nearby_venues(22.05, -159.30, 25000, 5000)

In [ ]:
venues_kauai[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-5591de2d498e50e3a5fb42a7-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/streetfood_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1cb941735',
    'name': 'Food Truck',
    'pluralName': 'Food Trucks',
    'primary': True,
    'shortName': 'Food Truck'}],
  'id': '5591de2d498e50e3a5fb42a7',
  'location': {'address': '4-1306 Kuhio Hwy',
   'cc': 'US',
   'city': 'Kapa‘a',
   'country': 'United States',
   'distance': 2620,
   'formattedAddress': ['4-1306 Kuhio Hwy',
    'Kapa‘a, HI 96746',
    'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 22.062575900926912,
     'lng': -159.3214723718646}],
   'lat': 22.062575900926912,
   'lng': -159.3214723718646,
   'postalCode': '96746',
   'state': 'HI'},
  'name': 'Wailua Shave Ice Kauai',
  'photos': {'cou

In [ ]:
# normalize JSON to dataframe
nearby_kauai0 = pd.json_normalize(venues_kauai)
nearby_kauai0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 27 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   referralId                           96 non-null     object 
 1   reasons.count                        96 non-null     int64  
 2   reasons.items                        96 non-null     object 
 3   venue.id                             96 non-null     object 
 4   venue.name                           96 non-null     object 
 5   venue.location.address               64 non-null     object 
 6   venue.location.lat                   96 non-null     float64
 7   venue.location.lng                   96 non-null     float64
 8   venue.location.labeledLatLngs        96 non-null     object 
 9   venue.location.distance              96 non-null     int64  
 10  venue.location.postalCode            74 non-null     object 
 11  venue.location.cc                 

In [ ]:
nearby_kauai0.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.venuePage.id,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name
0,e-0-5591de2d498e50e3a5fb42a7-0,0,"[{'summary': 'This spot is popular', 'type': '...",5591de2d498e50e3a5fb42a7,Wailua Shave Ice Kauai,4-1306 Kuhio Hwy,22.062576,-159.321472,"[{'label': 'display', 'lat': 22.06257590092691...",2620,96746,US,Kapa‘a,HI,United States,"[4-1306 Kuhio Hwy, Kapa‘a, HI 96746, United St...","[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e-0-560afea3498e26781b3d9bf6-1,0,"[{'summary': 'This spot is popular', 'type': '...",560afea3498e26781b3d9bf6,Nom Kauai,4-1620 Kuhio Highway,22.069152,-159.320067,"[{'label': 'display', 'lat': 22.0691518818603,...",2971,96746,US,Kapa‘a,HI,United States,"[4-1620 Kuhio Highway, Kapa‘a, HI 96746, Unite...","[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e-0-4b732119f964a5200f9d2de3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b732119f964a5200f9d2de3,Pono Market,4-1300 Kuhio Hwy,22.074320,-159.318922,"[{'label': 'display', 'lat': 22.07431990494844...",3337,96746,US,Kapa‘a,HI,United States,"[4-1300 Kuhio Hwy, Kapa‘a, HI 96746, United St...","[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e-0-52cda830498e6ac9b2ab9ed1-3,0,"[{'summary': 'This spot is popular', 'type': '...",52cda830498e6ac9b2ab9ed1,Kauai Juice Co,4-1384 Kuhio Highway,22.076290,-159.317842,"[{'label': 'display', 'lat': 22.07628963149579...",3457,96746,US,Kapa‘a,HI,United States,"[4-1384 Kuhio Highway, Kapa‘a, HI 96746, Unite...","[{'id': '4bf58dd8d48988d112941735', 'name': 'J...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e-0-5802f71538fa5e47444b5aa0-4,0,"[{'summary': 'This spot is popular', 'type': '...",5802f71538fa5e47444b5aa0,Kenji Burger,NaN,22.060731,-159.321456,"[{'label': 'display', 'lat': 22.06073068598406...",2515,96746,US,Kapa‘a,HI,United States,"[Kapa‘a, HI 96746, United States]","[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# duplicate dataframe
nearby_kauai1 = nearby_kauai0.copy()

In [ ]:
# filter columns,consider only required columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories'] + [col for col in nearby_kauai1.columns if col.startswith('venue.location.')]
nearby_kauai2 = nearby_kauai1.loc[:, filtered_columns]
nearby_kauai2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   venue.id                         96 non-null     object 
 1   venue.name                       96 non-null     object 
 2   venue.categories                 96 non-null     object 
 3   venue.location.address           64 non-null     object 
 4   venue.location.lat               96 non-null     float64
 5   venue.location.lng               96 non-null     float64
 6   venue.location.labeledLatLngs    96 non-null     object 
 7   venue.location.distance          96 non-null     int64  
 8   venue.location.postalCode        74 non-null     object 
 9   venue.location.cc                96 non-null     object 
 10  venue.location.city              85 non-null     object 
 11  venue.location.state             96 non-null     object 
 12  venue.location.country  

In [ ]:
# add an island column
nearby_kauai2['venue.island'] = 'Kauai'
nearby_kauai2['venue.county'] = 'Kauai'
nearby_kauai2.head()

,venue.id,venue.name,venue.categories,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.crossStreet,venue.island,venue.county
0,5591de2d498e50e3a5fb42a7,Wailua Shave Ice Kauai,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",4-1306 Kuhio Hwy,22.062576,-159.321472,"[{'label': 'display', 'lat': 22.06257590092691...",2620,96746,US,Kapa‘a,HI,United States,"[4-1306 Kuhio Hwy, Kapa‘a, HI 96746, United St...",NaN,Kauai,Kauai
1,560afea3498e26781b3d9bf6,Nom Kauai,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",4-1620 Kuhio Highway,22.069152,-159.320067,"[{'label': 'display', 'lat': 22.0691518818603,...",2971,96746,US,Kapa‘a,HI,United States,"[4-1620 Kuhio Highway, Kapa‘a, HI 96746, Unite...",NaN,Kauai,Kauai
2,4b732119f964a5200f9d2de3,Pono Market,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",4-1300 Kuhio Hwy,22.074320,-159.318922,"[{'label': 'display', 'lat': 22.07431990494844...",3337,96746,US,Kapa‘a,HI,United States,"[4-1300 Kuhio Hwy, Kapa‘a, HI 96746, United St...",NaN,Kauai,Kauai
3,52cda830498e6ac9b2ab9ed1,Kauai Juice Co,"[{'id': '4bf58dd8d48988d112941735', 'name': 'J...",4-1384 Kuhio Highway,22.076290,-159.317842,"[{'label': 'display', 'lat': 22.07628963149579...",3457,96746,US,Kapa‘a,HI,United States,"[4-1384 Kuhio Highway, Kapa‘a, HI 96746, Unite...",NaN,Kauai,Kauai
4,5802f71538fa5e47444b5aa0,Kenji Burger,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",NaN,22.060731,-159.321456,"[{'label': 'display', 'lat': 22.06073068598406...",2515,96746,US,Kapa‘a,HI,United States,"[Kapa‘a, HI 96746, United States]",NaN,Kauai,Kauai


### Niihau

In [ ]:
# fetch data from API
venues_niihau = get_nearby_venues(21.54, -160.10, 10000, 5000)

In [ ]:
venues_niihau

[]

### Kahoolawe

In [ ]:
# fetch data from API
venues_kahoolawe = get_nearby_venues(20.33, -156.36, 10000, 5000)

In [ ]:
venues_kahoolawe

[]

# Data Wrangling

In [ ]:
# Concatenate dataframes
nearby_venues0 = pd.concat([nearby_hawaii2, nearby_oahu2, nearby_maui2, nearby_molokai2, nearby_lanai2, nearby_kauai2], axis=0, ignore_index=True)
nearby_venues0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   venue.id                         255 non-null    object 
 1   venue.name                       255 non-null    object 
 2   venue.categories                 255 non-null    object 
 3   venue.location.lat               255 non-null    float64
 4   venue.location.lng               255 non-null    float64
 5   venue.location.labeledLatLngs    253 non-null    object 
 6   venue.location.distance          255 non-null    int64  
 7   venue.location.postalCode        211 non-null    object 
 8   venue.location.cc                255 non-null    object 
 9   venue.location.city              236 non-null    object 
 10  venue.location.state             250 non-null    object 
 11  venue.location.country           255 non-null    object 
 12  venue.location.formatt

In [ ]:
nearby_venues0.head()

,venue.id,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.address,venue.location.crossStreet,venue.island,venue.county,venue.location.neighborhood
0,4fc565dbe4b0829fe1507460,Lelekea Beach,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",20.651087,-156.079459,"[{'label': 'display', 'lat': 20.651087, 'lng':...",22815,96713,US,Hāna,HI,United States,"[Hāna, HI 96713, United States]",NaN,NaN,Hawaii,Hawaii,NaN
1,50f5e39ee4b0255d39ffc198,Haukini Bay,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",20.624844,-156.210364,"[{'label': 'display', 'lat': 20.62484351951365...",16159,96790,US,Kula,HI,United States,"[Kula, HI 96790, United States]",NaN,NaN,Hawaii,Hawaii,NaN
2,4e4e09abbd4101d0d7a0c8b7,Cold Stone Creamery,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",20.444210,-156.163382,"[{'label': 'display', 'lat': 20.44421, 'lng': ...",5518,96740,US,Kailua-Kona,HI,United States,"[75-1027 Henry St, Kailua-Kona, HI 96740, Unit...",75-1027 Henry St,NaN,Hawaii,Hawaii,NaN
3,501a833fe4b0a724be572f34,Alenuihaha Channel,"[{'id': '4bf58dd8d48988d12d951735', 'name': 'B...",20.390015,-156.246970,"[{'label': 'display', 'lat': 20.39001493552374...",11151,NaN,US,NaN,NaN,United States,[United States],NaN,NaN,Hawaii,Hawaii,NaN
4,57eb1a3d498e11e2a0e8c6a4,Pokowai Sea Arch,"[{'id': '4bf58dd8d48988d165941735', 'name': 'S...",20.622090,-156.213070,"[{'label': 'display', 'lat': 20.62209, 'lng': ...",15875,96790,US,Kula,HI,United States,"[Kula, HI 96790, United States]",NaN,NaN,Hawaii,Hawaii,NaN


In [ ]:
# filter the category for each row
nearby_venues0['venue.categories'] = nearby_venues0.apply(get_category_type, axis=1)

In [ ]:
# clean columns
nearby_venues0.columns = [col.split('.')[-1] for col in nearby_venues0.columns]
nearby_venues0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                255 non-null    object 
 1   name              255 non-null    object 
 2   categories        255 non-null    object 
 3   lat               255 non-null    float64
 4   lng               255 non-null    float64
 5   labeledLatLngs    253 non-null    object 
 6   distance          255 non-null    int64  
 7   postalCode        211 non-null    object 
 8   cc                255 non-null    object 
 9   city              236 non-null    object 
 10  state             250 non-null    object 
 11  country           255 non-null    object 
 12  formattedAddress  255 non-null    object 
 13  address           186 non-null    object 
 14  crossStreet       61 non-null     object 
 15  island            255 non-null    object 
 16  county            255 non-null    object 
 1

In [ ]:
nearby_venues0.head()

,id,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,crossStreet,island,county,neighborhood
0,4fc565dbe4b0829fe1507460,Lelekea Beach,Beach,20.651087,-156.079459,"[{'label': 'display', 'lat': 20.651087, 'lng':...",22815,96713,US,Hāna,HI,United States,"[Hāna, HI 96713, United States]",NaN,NaN,Hawaii,Hawaii,NaN
1,50f5e39ee4b0255d39ffc198,Haukini Bay,Beach,20.624844,-156.210364,"[{'label': 'display', 'lat': 20.62484351951365...",16159,96790,US,Kula,HI,United States,"[Kula, HI 96790, United States]",NaN,NaN,Hawaii,Hawaii,NaN
2,4e4e09abbd4101d0d7a0c8b7,Cold Stone Creamery,Ice Cream Shop,20.444210,-156.163382,"[{'label': 'display', 'lat': 20.44421, 'lng': ...",5518,96740,US,Kailua-Kona,HI,United States,"[75-1027 Henry St, Kailua-Kona, HI 96740, Unit...",75-1027 Henry St,NaN,Hawaii,Hawaii,NaN
3,501a833fe4b0a724be572f34,Alenuihaha Channel,Boat or Ferry,20.390015,-156.246970,"[{'label': 'display', 'lat': 20.39001493552374...",11151,NaN,US,NaN,NaN,United States,[United States],NaN,NaN,Hawaii,Hawaii,NaN
4,57eb1a3d498e11e2a0e8c6a4,Pokowai Sea Arch,Scenic Lookout,20.622090,-156.213070,"[{'label': 'display', 'lat': 20.62209, 'lng': ...",15875,96790,US,Kula,HI,United States,"[Kula, HI 96790, United States]",NaN,NaN,Hawaii,Hawaii,NaN


In [ ]:
# export reviews dataset
nearby_venues0.to_csv('/content/drive/My Drive/CIS550_Project/Data/nearby_venues0.csv', index=True, line_terminator='\n')

# Data Visualization

In [ ]:
# specify parameters
latitude = 21.30
longitude = -157.86

In [ ]:
# creating map zoom at present Location
map = folium.Map(location=[latitude,longitude],zoom_start=14)

In [ ]:
# adding the current location to map
folium.CircleMarker(location=[latitude,longitude],radius=5,color='red',fill=True,fill_color='red').add_to(map)
folium.Marker(location=[latitude,longitude],popup="<strong> Current Location</strong>",icon = folium.Icon(color='red')).add_to(map)

In [ ]:
# drawing radius of search
folium.Circle(location=[latitude,longitude],radius=radius,color='blue',opacity=0.5,fill=True,fill_color='blue').add_to(map)

In [ ]:
# adding all the popular places around
for lat,lon,name,cate,addr,dist in zip(nearby2.lat,nearby2.lng,nearby2.name,nearby2.categories,nearby2.address,nearby2.distance):
  folium.CircleMarker(location=[lat,lon],radius=5,color="red",fill=True,fill_color='red').add_to(map)
  folium.Marker(location=[lat,lon],
              popup= folium.Popup(('<strong><font color ="red">Name :     </font>'+name+'</strong><br>'+
                                   '<strong><font color ="red">Categorie :    </font>'+cate+'</strong><br>'+
                                   '<strong><font color ="red">Address :    </font>'+addr+'</strong><br>'+
                                   '<strong><font color ="red">Distance :    </font>'+str(dist)+' mts</strong>'
                                   ),max_width=250)).add_to(map)

In [ ]:
map